In [1]:
import cdsodatacli.query as qr
import datetime
import requests
import pandas as pd
import argparse
from shapely.geometry import GeometryCollection, LineString, Point, Polygon, MultiPolygon
import geopandas as gpd
import shapely
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pytz
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import warnings
import logging 
import xarray as xr
import rasterio 
from affine import Affine
import os

In [2]:
from importlib import reload
reload(logging)
logging.basicConfig(level=logging.DEBUG)

In [3]:
gdf = "/tmp/GL_TS_DB_6401850.nc"
netcdf_data = qr.fetch_data(gdf=gdf, cache_dir='/home1/datahome/jrmiadan/cache_dir/netcdf_cache', figure=True )

DEBUG:root:normalize_gdf processing time:0.025386333465576172s
DEBUG:root:Length of input:3190
DEBUG:root:creat_urls processing time:0.5008223056793213s
  0%|                                          | 1/3190 [00:00<20:16,  2.62it/s]DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
  0%|                                          | 6/3190 [00:00<06:59,  7.59it/s]DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
  0%|                                          | 8/3190 [00:00<05:41,  9.32it/s]DEBUG:root:{'@odata.context': '$metadata#Products(Attributes())', 'value': []}
DEBUG:root:{'@odata.context': '$metadata#Products(Attr

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [42]:
gdf = "/tmp/GL_TS_DB_6401850.nc"
ds = xr.open_dataset(gdf)
df = pd.DataFrame()
df['LONGITUDE'] = ds['LONGITUDE']
df['LATITUDE'] = ds['LATITUDE']

geometry = gpd.points_from_xy(df.LONGITUDE, df.LATITUDE)
gdf = gpd.GeoDataFrame(df, geometry=geometry)
collection = 'SENTINEL-1'
# Add time column
gdf['time'] = pd.to_datetime(ds['TIME'])
gdf['start_datetime'] = gdf.time - datetime.timedelta(hours=10)
gdf['end_datetime'] = gdf.time + datetime.timedelta(hours=10)
gdf['collection'] = [collection for xx in range(len(gdf['time']))]
gdf
gdf_norm = qr.normalize_gdf(gdf)
urls = qr.urls_creat(gdf_norm)
urls

DEBUG:root:normalize_gdf processing time:0.02716827392578125s
DEBUG:root:creat_urls processing time:0.46532487869262695s


[(0,
  "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=OData.CSC.Intersects(area=geography'SRID=4326;POINT(-18.491899490356445%2061.726768493652344)') and Collection/Name eq 'SENTINEL-1' and ContentDate/Start gt 2021-03-01T04:39:59.000Z and ContentDate/Start lt 2021-03-02T00:39:59.000Z&$top=1000&$expand=Attributes"),
 (1,
  "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=OData.CSC.Intersects(area=geography'SRID=4326;POINT(-18.495529174804688%2061.726280212402344)') and Collection/Name eq 'SENTINEL-1' and ContentDate/Start gt 2021-03-01T05:00:00.000Z and ContentDate/Start lt 2021-03-02T01:00:00.000Z&$top=1000&$expand=Attributes"),
 (2,
  "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=OData.CSC.Intersects(area=geography'SRID=4326;POINT(-18.508190155029297%2061.7246208190918)') and Collection/Name eq 'SENTINEL-1' and ContentDate/Start gt 2021-03-01T06:01:00.000Z and ContentDate/Start lt 2021-03-02T02:01:00.000Z&$top=1000&$exp

In [43]:
import hashlib

def get_cache_filename(url, cache_dir=None):
    """
    return filename from the cache
    """
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()
    return os.path.join(cache_dir, url_hash + '.json')


def fetch_data_from_url(url, index, cache_dir=None):
    """
    return data fetched from url
    """
    cache_file = get_cache_filename(url, cache_dir)
    print(cache_file)
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            data = json.load(f)
    else:
        response = requests.get(url)
        data = response.json()
        if 'value' in data and len(data['value']) > 0:
            with open(cache_file, 'w') as f:
                json.dump(data, f)
    # process data
    df = pd.DataFrame.from_dict(data['value'])
    df['input_index'] = index
    return df


def fetch_data_from_urls(urls, cache_dir=None, max_workers=50):
    """
    return all data fetched from urls
    """
    collected_data = pd.DataFrame()

    with ThreadPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(urls)) as pbar:
        future_to_url = {executor.submit(fetch_data_from_url, url, index, cache_dir): (url, index) for index, url in
                         urls}
        for future in as_completed(future_to_url):
            df = future.result()
            collected_data = pd.concat([collected_data, df])
            pbar.update(1)
    return collected_data

In [44]:
urls1 = urls[:10]
data = fetch_data_from_urls(urls1, cache_dir='/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test', max_workers=50)
data

  0%|                                                    | 0/10 [00:00<?, ?it/s]

/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/a4329980b4de53ca0a195d500ef8107a.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/6a0edf1908b03c4ca4250f3c9203cb5e.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/35cc96e5515d1b1693d234059957c39c.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/226801ece289d53c7e204c10136ba2d3.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/708991cc92a81fcf49fed3874ad19e65.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/5919f16ef667e40a38f1febcbf21007e.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/c9efd0a77dd38916277f8751fdd025c4.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/73852057b2a845f85ca44149c1adf9f4.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/3b8ecf6b8a8d6f67d4594277b502a52f.json
/home1/datahome/jrmiadan/cache_dir/netcdf_cache_test/656f3ef26f613c7b09201950fc287c99.json


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 15.51it/s]


,input_index


In [ ]:
netcdf_data = qr.fetch_data(gdf=gdf, cache_dir='/home1/datahome/jrmiadan/cache_dir/netcdf_cache', figure=True )

In [45]:
json = requests.get("https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=OData.CSC.Intersects(area=geography'SRID=4326;POINT(-18.491899490356445%2061.726768493652344)') and ContentDate/Start gt 2021-03-01T04:39:59.000Z and ContentDate/Start lt 2021-03-01T04:39:59.000Z&$top=1000").json()
df = pd.DataFrame.from_dict(json['value'])
df

""
